In [2]:
# Thêm autoreload vào để tự động reload lại module nếu có thay đổi code trong module
%load_ext autoreload
%autoreload 2

import sys
import os

# Thêm đường dẫn dẫn đến thư mục cha
sys.path.append(os.path.abspath(".."))

In [ ]:
# Check các đường dẫn mà có thể truy cập từ file này
sys.path

In [3]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [4]:
from common.utils import extract_parts_of_file, get_current_time_string

In [16]:
def extract_key_points_from_image(image_path, label, file_output, is_save_pose_image=False):
    file_name, file_extension = extract_parts_of_file(image_path)
    current_time = get_current_time_string()

    # Kiểm tra và tạo thư mục nếu chưa tồn tại
    output_directory = "result"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Khởi tạo MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, smooth_landmarks=True)
    mp_drawing = mp.solutions.drawing_utils

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Phát hiện pose
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        columns_name = ["label"]
        columns_value = [label]
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            land_mark_name = mp_pose.PoseLandmark(id).name.lower()
            columns_name += [
                f"{ land_mark_name }_x",
                f"{ land_mark_name }_y",
                f"{ land_mark_name }_z",
            ]

            columns_value += [
                landmark.x * image_width,
                landmark.y * image_height,
                landmark.z * image_width,
            ]

        # Vẽ keypoints lên ảnh
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    df_keypoints = pd.DataFrame([columns_value], columns=columns_name)

    with open(f"{output_directory}/{file_output}", 'a') as f:
        # header=f.tell()==0 sẽ ghi tiêu đề nếu file trống
        df_keypoints.to_csv(f, header=f.tell() == 0, index=False)

    # Lưu ảnh đã vẽ keypoints vào file mới
    if is_save_pose_image:
        images_folder = f"{output_directory}/images"
        if not os.path.exists(images_folder):
            os.makedirs(images_folder)

        cv2.imwrite(
            f"{images_folder}/{current_time}_{file_name}.{file_extension}", image
        )

    pose.close()

In [17]:
extract_key_points_from_image("Lunge.png", "C", "train.csv", is_save_pose_image=True)